In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
from utils import DogsDataset, preprocessing_transfert, data_augment_transfert

In [3]:
# 📂 Chargement des DataFrames sauvegardés
df_train = pd.read_pickle("df_train.pkl")
df_test = pd.read_pickle("df_test.pkl")

In [4]:
preprocessing_transfert_train = DogsDataset(df_train, transforms=preprocessing_transfert)
preprocessing_transfert_test = DogsDataset(df_test, transforms=preprocessing_transfert)

In [5]:
augmentation_transfert_train = DogsDataset(df_train, transforms=data_augment_transfert)
augmentation_transfert_test = DogsDataset(df_test, transforms=data_augment_transfert)

In [6]:
from torch.utils.data import DataLoader
from tqdm import tqdm

# Assurez-vous que votre DataLoader est correctement configuré
transfert_train_loader = DataLoader(preprocessing_transfert_train, batch_size=32, shuffle=True)
transfert_test_loader = DataLoader(preprocessing_transfert_test, batch_size=32, shuffle=False)

In [7]:
# Récupérer un batch d'images
data_iter = iter(transfert_train_loader)
images, labels = next(data_iter)

# Vérifier la taille des images
print(f"Taille du batch : {images.shape}")  # Doit être (batch_size, 3, 224, 224)

Taille du batch : torch.Size([32, 3, 224, 224])


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder

# 📌 Charger VGG16 pré-entraîné
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_transfert = models.vgg16(pretrained=True)

# 🔄 Modifier la dernière couche (10 classes)
num_classes = 10
model_transfert.classifier[6] = nn.Linear(in_features=4096, out_features=num_classes)

# ⚙️ Geler les couches convolutionnelles
for param in model_transfert.features.parameters():
    param.requires_grad = False

model_transfert = model_transfert.to(device)

# 🎯 Définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_transfert.classifier.parameters(), lr=0.0001)


c:\Users\mathi\miniconda3\envs\pytorch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mathi\miniconda3\envs\pytorch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
# 🚀 Entraînement du modèle
num_epochs = 20

# Dossier de sauvegarde
checkpoint_dir = "./checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

# Initialisation
best_accuracy = -float('inf')  # ou float('inf') pour suivre la loss
best_epoch = -1

for epoch in range(num_epochs):
    model_transfert.train()
    train_loss = 0.0

    with tqdm(transfert_train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as tepoch:
      for inputs, labels in tepoch:
        inputs = inputs.to(device)
        labels = labels.to(device) if isinstance(labels, torch.Tensor) else torch.tensor(labels, dtype=torch.long, device=device)

        optimizer.zero_grad()  # Remet à zéro les gradients de l'optimiseur

        outputs = model_transfert(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        tepoch.set_postfix(loss=train_loss / (tepoch.n + 1))

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss/len(transfert_train_loader):.4f}")

    # 🔎 Évaluation
    model_transfert.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in transfert_test_loader:
          inputs = inputs.to(device)
          labels = labels.to(device) if isinstance(labels, torch.Tensor) else torch.tensor(labels, dtype=torch.long, device=device)

          outputs = model_transfert(inputs)
          loss = criterion(outputs, labels)

          val_loss += loss.item()
          _, predicted = outputs.max(1)
          correct += (predicted == labels).sum().item()
          total += labels.size(0)

    val_accuracy = 100 * correct / total
    print(f"Validation Loss: {val_loss / len(transfert_test_loader):.4f}, Accuracy: {val_accuracy:.2f}%")

    # Sauvegarde si le modèle est meilleur
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_epoch = epoch + 1

        torch.save(model_transfert.state_dict(), os.path.join(checkpoint_dir, "best_model_transfert.pth"))
        print(f"✅ Nouveau meilleur modèle sauvegardé à l'epoch {best_epoch} (accuracy = {best_accuracy:.4f})")

# Info finale
print(f"\n✅ Meilleur modèle obtenu à l'epoch {best_epoch} avec accuracy = {best_accuracy:.4f}")

Epoch 1/20: 100%|██████████| 56/56 [00:14<00:00,  3.75batch/s, loss=0.619]


Epoch 1/20, Loss: 0.6189
Validation Loss: 0.2862, Accuracy: 92.00%
✅ Nouveau meilleur modèle sauvegardé à l'epoch 1 (accuracy = 92.0000)


Epoch 2/20: 100%|██████████| 56/56 [00:14<00:00,  3.88batch/s, loss=0.0962]


Epoch 2/20, Loss: 0.0962
Validation Loss: 0.2662, Accuracy: 92.67%
✅ Nouveau meilleur modèle sauvegardé à l'epoch 2 (accuracy = 92.6667)


Epoch 3/20: 100%|██████████| 56/56 [00:14<00:00,  3.87batch/s, loss=0.0402]


...

Epoch 18/20, Loss: 0.0019
Validation Loss: 0.6114, Accuracy: 90.67%


Epoch 19/20: 100%|██████████| 56/56 [00:14<00:00,  3.76batch/s, loss=0.0057] 


Epoch 19/20, Loss: 0.0057
Validation Loss: 1.2538, Accuracy: 84.67%


Epoch 20/20: 100%|██████████| 56/56 [00:14<00:00,  3.81batch/s, loss=0.023]  


Epoch 20/20, Loss: 0.0230
Validation Loss: 0.8076, Accuracy: 88.00%

✅ Meilleur modèle obtenu à l'epoch 5 avec accuracy = 94.0000


In [10]:
# Assurez-vous que votre DataLoader est correctement configuré
transfert_augment_train_loader = DataLoader(augmentation_transfert_train, batch_size=32, shuffle=True)
transfert_augment_test_loader = DataLoader(augmentation_transfert_test, batch_size=32, shuffle=False)

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder

# 📌 Charger VGG16 pré-entraîné
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_transfert_augment = models.vgg16(pretrained=True)

# 🔄 Modifier la dernière couche (10 classes)
num_classes = 10
model_transfert_augment.classifier[6] = nn.Linear(in_features=4096, out_features=num_classes)

# ⚙️ Geler les couches convolutionnelles
for param in model_transfert_augment.features.parameters():
    param.requires_grad = False

model_transfert_augment = model_transfert_augment.to(device)

# 🎯 Définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_transfert_augment.classifier.parameters(), lr=0.0001)


In [12]:
# 🚀 Entraînement du modèle
num_epochs = 20

# Dossier de sauvegarde
checkpoint_dir = "./checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

# Initialisation
best_accuracy = -float('inf')  # ou float('inf') pour suivre la loss
best_epoch = -1


for epoch in range(num_epochs):
    model_transfert_augment.train()
    train_loss = 0.0

    with tqdm(transfert_augment_train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as tepoch:
      for inputs, labels in tepoch:
        inputs = inputs.to(device)
        labels = labels.to(device) if isinstance(labels, torch.Tensor) else torch.tensor(labels, dtype=torch.long, device=device)

        optimizer.zero_grad()  # Remet à zéro les gradients de l'optimiseur

        outputs = model_transfert_augment(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        tepoch.set_postfix(loss=train_loss / (tepoch.n + 1))

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss/len(transfert_augment_train_loader):.4f}")

    # 🔎 Évaluation
    model_transfert_augment.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in transfert_augment_test_loader:
          inputs = inputs.to(device)
          labels = labels.to(device) if isinstance(labels, torch.Tensor) else torch.tensor(labels, dtype=torch.long, device=device)

          outputs = model_transfert_augment(inputs)
          loss = criterion(outputs, labels)

          val_loss += loss.item()
          _, predicted = outputs.max(1)
          correct += (predicted == labels).sum().item()
          total += labels.size(0)

    val_accuracy = 100 * correct / total
    print(f"Validation Loss: {val_loss / len(transfert_augment_test_loader):.4f}, Accuracy: {val_accuracy:.2f}%")

    # Sauvegarde si le modèle est meilleur
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_epoch = epoch + 1

        torch.save(model_transfert_augment.state_dict(), os.path.join(checkpoint_dir, "best_model_transfert_augment.pth"))
        print(f"✅ Nouveau meilleur modèle sauvegardé à l'epoch {best_epoch} (accuracy = {best_accuracy:.4f})")

# Info finale
print(f"\n✅ Meilleur modèle obtenu à l'epoch {best_epoch} avec accuracy = {best_accuracy:.4f}")

Epoch 1/20: 100%|██████████| 56/56 [00:15<00:00,  3.61batch/s, loss=0.565]


Epoch 1/20, Loss: 0.5654
Validation Loss: 0.2545, Accuracy: 90.67%
✅ Nouveau meilleur modèle sauvegardé à l'epoch 1 (accuracy = 90.6667)


Epoch 2/20: 100%|██████████| 56/56 [00:15<00:00,  3.62batch/s, loss=0.207]


Epoch 2/20, Loss: 0.2070
Validation Loss: 0.2660, Accuracy: 90.00%


Epoch 3/20: 100%|██████████| 56/56 [00:15<00:00,  3.61batch/s, loss=0.133]


...

Epoch 18/20, Loss: 0.0340
Validation Loss: 0.4707, Accuracy: 91.33%


Epoch 19/20: 100%|██████████| 56/56 [00:15<00:00,  3.61batch/s, loss=0.0337] 


Epoch 19/20, Loss: 0.0337
Validation Loss: 0.4484, Accuracy: 91.33%


Epoch 20/20: 100%|██████████| 56/56 [00:15<00:00,  3.60batch/s, loss=0.0246]


Epoch 20/20, Loss: 0.0246
Validation Loss: 0.5314, Accuracy: 92.00%

✅ Meilleur modèle obtenu à l'epoch 14 avec accuracy = 94.0000
